### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [8]:
# Dependencies and Setup
import pandas as pd
import numpy as np

# File to Load (Remember to Change These)
school_data_to_load = "Resources/schools_complete.csv"
student_data_to_load = "Resources/students_complete.csv"

# Read School and Student Data File and store into Pandas DataFrames
school_data = pd.read_csv(school_data_to_load)
student_data = pd.read_csv(student_data_to_load)

# Combine the data into a single dataset.  
school_data_complete = pd.merge(student_data, school_data, how="left", on=["school_name", "school_name"])

## District Summary

* Calculate the total number of schools

* Calculate the total number of students

* Calculate the total budget

* Calculate the average math score 

* Calculate the average reading score

* Calculate the percentage of students with a passing math score (70 or greater)

* Calculate the percentage of students with a passing reading score (70 or greater)

* Calculate the percentage of students who passed math **and** reading (% Overall Passing)

* Create a dataframe to hold the above results

* Optional: give the displayed data cleaner formatting

In [9]:
num_schools = school_data["school_name"].count()
student_tot = school_data_complete["student_name"].count()
budg_tot = school_data["budget"].sum()
avg_math = round(school_data_complete["math_score"].mean(),2)
avg_read = round(school_data_complete["reading_score"].mean(),2)
#making pass bin fails for passing percentage calculations
pass_bin = [0,70,100]
pass_labels = ["Fail","Pass"]
school_data_complete["read_res"] = pd.cut(school_data_complete["reading_score"], pass_bin, labels=pass_labels)
school_data_complete["math_res"] = pd.cut(school_data_complete["math_score"], pass_bin, labels=pass_labels)
#calculating the passing for each subject
read_pass_count = (school_data_complete["read_res"] == "Pass").sum()
read_pass_perc = f"{round((read_pass_count / student_tot) *100, 2)}%"
math_pass_count = (school_data_complete["math_res"] == "Pass").sum()
math_pass_perc = f"{round((math_pass_count / student_tot) *100, 2)}%"
combo_pass_count = ((school_data_complete["read_res"] == "Pass") & (school_data_complete["math_res"] == "Pass")).sum()
combo_pass_perc = f"{round((combo_pass_count / student_tot) *100, 2)}%"
#formatting
dist_summ_df = pd.DataFrame([[num_schools,student_tot,budg_tot,avg_math,math_pass_perc,avg_read,read_pass_perc,combo_pass_perc]], columns=["Total Schools","Total Students","Total Budget","Average Math Score","% Passing Math","Average Reading Score","% Passing Reading","% Overall Passing"])


dist_summ_df

,Total Schools,Total Students,Total Budget,Average Math Score,% Passing Math,Average Reading Score,% Passing Reading,% Overall Passing
0,15,39170,24649428,78.99,72.39%,81.88,82.97%,60.8%


## School Summary

* Create an overview table that summarizes key metrics about each school, including:
  * School Name
  * School Type
  * Total Students
  * Total School Budget
  * Per Student Budget
  * Average Math Score
  * Average Reading Score
  * % Passing Math
  * % Passing Reading
  * % Overall Passing (The percentage of students that passed math **and** reading.)
  
* Create a dataframe to hold the above results

In [10]:
#setting initial data frames to work off of
group_schools = school_data_complete.groupby("school_name")
group_schools_2 = school_data_complete.set_index("school_name")
#pulling specific data to use in making columns
    #type and budget columns data
group_sch_set_vars = group_schools_2.loc[ : , ["type", "budget"]]
    #percentage of passing student data
pass_fail_read = group_schools_2.loc[group_schools_2["read_res"] != "Fail"]
pass_fail_math = group_schools_2.loc[group_schools_2["math_res"] != "Fail"]
pass_fail_combo =  group_schools_2.loc[(group_schools_2["math_res"] != "Fail") & (group_schools_2["read_res"] != "Fail")]
read_pass_ss = pass_fail_read.groupby("school_name").agg(read_pass_count = pd.NamedAgg(column="student_name", aggfunc="count"))
math_pass_ss = pass_fail_math.groupby("school_name").agg(math_pass_count = pd.NamedAgg(column="student_name", aggfunc="count"))
combo_pass_ss = pass_fail_combo.groupby("school_name").agg(combo_pass_count = pd.NamedAgg(column="student_name", aggfunc="count"))
    #average of scores data
group_sch_avgs = group_schools[["math_score", "reading_score"]].mean()
    #student count data
group_sch_stu_count = group_schools.agg(Total_Students = pd.NamedAgg(column="student_name", aggfunc="count"))
#merging all data into one data frame
mer_sch_summ = pd.merge(pd.merge(group_sch_avgs,group_sch_stu_count, on="school_name"),group_sch_set_vars, on="school_name", how="left").drop_duplicates()
mer_sch_summ["Per Student Budget"] = mer_sch_summ["budget"]/mer_sch_summ["Total_Students"]
mer_sch_summ["% Passing Math"] = math_pass_ss["math_pass_count"] / group_sch_stu_count["Total_Students"]
mer_sch_summ["% Passing Reading"] = read_pass_ss["read_pass_count"] / group_sch_stu_count["Total_Students"]
mer_sch_summ["% Overall Passing"] = combo_pass_ss["combo_pass_count"] / group_sch_stu_count["Total_Students"]
#Formatting
#clean_Sch_Summ = pd.DataFrame([[group_sch_set_vars[["type"]],group_sch_stu_count[["Total_Students"]],group_sch_set_vars[["budget"]],mer_sch_summ[["Per Student Budget"]],group_sch_avgs[["math_score", "reading_score"]],mer_sch_summ[["% Passing Math","% Passing Reading","% Overall Passing"]]]], columns=["School Type","Total Students","Total School Budget","Per Student Budget","Average Math Score","Average Reading Score","% Passing Math","% Passing Reading","% Overall Passing"])


mer_sch_summ

,math_score,reading_score,Total_Students,type,budget,Per Student Budget,% Passing Math,% Passing Reading,% Overall Passing
school_name,,,,,,,,,
Bailey High School,77.048432,81.033963,4976,District,3124928,628.0,0.646302,0.793006,0.511455
Cabrera High School,83.061895,83.975780,1858,Charter,1081356,582.0,0.895587,0.938644,0.840151
Figueroa High School,76.711767,81.158020,2949,District,1884411,639.0,0.637504,0.784334,0.499152
Ford High School,77.102592,80.746258,2739,District,1763916,644.0,0.657539,0.775100,0.512961
Griffin High School,83.351499,83.816757,1468,Charter,917500,625.0,0.897139,0.933924,0.836512
Hernandez High School,77.289752,80.934412,4635,District,3022020,652.0,0.647465,0.781877,0.501618
Holden High School,83.803279,83.814988,427,Charter,248087,581.0,0.906323,0.927400,0.840749
Huang High School,76.629414,81.182722,2917,District,1910635,655.0,0.633185,0.788138,0.499143
Johnson High School,77.072464,80.966394,4761,District,3094650,650.0,0.638521,0.782819,0.498005


In [11]:
## Top Performing Schools (By % Overall Passing)

* Sort and display the top five performing schools by % overall passing.

In [12]:
mer_sch_summ.sort_values("% Overall Passing",ascending=False).head()

,math_score,reading_score,Total_Students,type,budget,Per Student Budget,% Passing Math,% Passing Reading,% Overall Passing
school_name,,,,,,,,,
Wilson High School,83.274201,83.989488,2283,Charter,1319574,578.0,0.909330,0.932545,0.848883
Pena High School,83.839917,84.044699,962,Charter,585858,609.0,0.916840,0.922037,0.848233
Wright High School,83.682222,83.955000,1800,Charter,1049400,583.0,0.902778,0.934444,0.844444
Thomas High School,83.418349,83.848930,1635,Charter,1043130,638.0,0.902141,0.929052,0.842813
Holden High School,83.803279,83.814988,427,Charter,248087,581.0,0.906323,0.927400,0.840749


## Bottom Performing Schools (By % Overall Passing)

* Sort and display the five worst-performing schools by % overall passing.

In [13]:
mer_sch_summ.sort_values("% Overall Passing").head()

,math_score,reading_score,Total_Students,type,budget,Per Student Budget,% Passing Math,% Passing Reading,% Overall Passing
school_name,,,,,,,,,
Rodriguez High School,76.842711,80.744686,3999,District,2547363,637.0,0.640660,0.777444,0.494374
Johnson High School,77.072464,80.966394,4761,District,3094650,650.0,0.638521,0.782819,0.498005
Huang High School,76.629414,81.182722,2917,District,1910635,655.0,0.633185,0.788138,0.499143
Figueroa High School,76.711767,81.158020,2949,District,1884411,639.0,0.637504,0.784334,0.499152
Hernandez High School,77.289752,80.934412,4635,District,3022020,652.0,0.647465,0.781877,0.501618


## Math Scores by Grade

* Create a table that lists the average Reading Score for students of each grade level (9th, 10th, 11th, 12th) at each school.

  * Create a pandas series for each grade. Hint: use a conditional statement.
  
  * Group each series by school
  
  * Combine the series into a dataframe
  
  * Optional: give the displayed data cleaner formatting

In [19]:
ninth = group_schools_2.query("grade=='9th'")
ninth = ninth.filter(["school_name", "math_score"])
math_sco_grade = ninth.groupby(["school_name"]).mean()
math_sco_grade = math_sco_grade.rename(columns={"school_name":"School Name","math_score":"9th"})
tenth = group_schools_2.query("grade=='10th'")
tenth = tenth.filter(["school_name", "math_score"])
math_sco_grade["10th"] = tenth.groupby(["school_name"]).mean()
eleventh = group_schools_2.query("grade=='11th'")
eleventh = eleventh.filter(["school_name", "math_score"])
math_sco_grade["11th"] = eleventh.groupby(["school_name"]).mean()
twelth = group_schools_2.query("grade=='12th'")
twelth = twelth.filter(["school_name", "math_score"])
math_sco_grade["12th"] = twelth.groupby(["school_name"]).mean()


#school_data_complete.head()
math_sco_grade

,9th,10th,11th,12th
school_name,,,,
Bailey High School,77.083676,76.996772,77.515588,76.492218
Cabrera High School,83.094697,83.154506,82.765560,83.277487
Figueroa High School,76.403037,76.539974,76.884344,77.151369
Ford High School,77.361345,77.672316,76.918058,76.179963
Griffin High School,82.044010,84.229064,83.842105,83.356164
Hernandez High School,77.438495,77.337408,77.136029,77.186567
Holden High School,83.787402,83.429825,85.000000,82.855422
Huang High School,77.027251,75.908735,76.446602,77.225641
Johnson High School,77.187857,76.691117,77.491653,76.863248


## Reading Score by Grade 

* Perform the same operations as above for reading scores

In [23]:
ninth = group_schools_2.query("grade=='9th'")
ninth = ninth.filter(["school_name", "reading_score"])
read_sco_grade = ninth.groupby(["school_name"]).mean()
read_sco_grade = read_sco_grade.rename(columns={"school_name":"School Name","reading_score":"9th"})
tenth = group_schools_2.query("grade=='10th'")
tenth = tenth.filter(["school_name", "reading_score"])
read_sco_grade["10th"] = tenth.groupby(["school_name"]).mean()
eleventh = group_schools_2.query("grade=='11th'")
eleventh = eleventh.filter(["school_name", "reading_score"])
read_sco_grade["11th"] = eleventh.groupby(["school_name"]).mean()
twelth = group_schools_2.query("grade=='12th'")
twelth = twelth.filter(["school_name", "reading_score"])
read_sco_grade["12th"] = twelth.groupby(["school_name"]).mean()

read_sco_grade

,9th,10th,11th,12th
school_name,,,,
Bailey High School,81.303155,80.907183,80.945643,80.912451
Cabrera High School,83.676136,84.253219,83.788382,84.287958
Figueroa High School,81.198598,81.408912,80.640339,81.384863
Ford High School,80.632653,81.262712,80.403642,80.662338
Griffin High School,83.369193,83.706897,84.288089,84.013699
Hernandez High School,80.866860,80.660147,81.396140,80.857143
Holden High School,83.677165,83.324561,83.815534,84.698795
Huang High School,81.290284,81.512386,81.417476,80.305983
Johnson High School,81.260714,80.773431,80.616027,81.227564


## Scores by School Spending

* Create a table that breaks down school performances based on average Spending Ranges (Per Student). Use 4 reasonable bins to group school spending. Include in the table each of the following:
  * Average Math Score
  * Average Reading Score
  * % Passing Math
  * % Passing Reading
  * Overall Passing Rate (Average of the above two)

In [25]:
spend_bin = [0,585,630,645,680]
spend_labels = ["<$585","$585-630","$630-645","$645-680"]
mer_sch_summ["Spending Ranges (Per Student)"] = pd.cut(mer_sch_summ["Per Student Budget"], spend_bin, labels=spend_labels)
school_spend = mer_sch_summ.filter(["Spending Ranges (Per Student)", "math_score", "reading_score", "% Passing Math", "% Passing Reading", "% Overall Passing"])
school_spend = school_spend.groupby(["Spending Ranges (Per Student)"]).mean()
school_spend

,math_score,reading_score,% Passing Math,% Passing Reading,% Overall Passing
Spending Ranges (Per Student),,,,,
<$585,83.455399,83.933814,0.903504,0.933258,0.843557
$585-630,81.899826,83.155286,0.839801,0.893786,0.757028
$630-645,78.518855,81.624473,0.709461,0.816483,0.587325
$645-680,76.997210,81.027843,0.639724,0.784278,0.499589


## Scores by School Size

* Perform the same operations as above, based on school size.

In [28]:
size_bin = [0,1000,2000,5000]
size_labels = ["Small(<1000)","Medium(1000-2000)","Large(2000-5000)"]
mer_sch_summ["School Size"] = pd.cut(mer_sch_summ["Total_Students"], size_bin, labels=size_labels)
school_size = mer_sch_summ.filter(["School Size", "math_score", "reading_score", "% Passing Math", "% Passing Reading", "% Overall Passing"])
school_size = school_size.groupby(["School Size"]).mean()
school_size

,math_score,reading_score,% Passing Math,% Passing Reading,% Overall Passing
School Size,,,,,
Small(<1000),83.821598,83.929843,0.911582,0.924719,0.844491
Medium(1000-2000),83.374684,83.864438,0.899313,0.932448,0.839167
Large(2000-5000),77.746417,81.344493,0.676313,0.801908,0.545699


## Scores by School Type

* Perform the same operations as above, based on school type

In [35]:
#type_bin = [0,70,100]
#type_labels = ["Fail","Pass"]

school_type = pd.pivot_table(mer_sch_summ, values=["math_score", "reading_score", "% Passing Math", "% Passing Reading", "% Overall Passing"], index="type", aggfunc=np.mean, sort=False )
school_type

,% Overall Passing,% Passing Math,% Passing Reading,math_score,reading_score
type,,,,,
District,0.502387,0.643025,0.783246,76.956733,80.966636
Charter,0.841712,0.903632,0.930528,83.473852,83.896421
